In [1]:
#Preparamos el entorno y sesión

import findspark
findspark.init()

import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("practica_bdp").getOrCreate()

In [2]:
# 1. ¿Cuál es el país más “feliz” del 2021 según la data? (considerar que la columna “Ladder score” mayor número más feliz es el país)

df = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report-2021.csv")

df.columns

df.select('Country name','Ladder score').sort(desc('Ladder score')).show(1)

+------------+------------+
|Country name|Ladder score|
+------------+------------+
|     Finland|       7.842|
+------------+------------+
only showing top 1 row



In [3]:
# 2. ¿Cuál es el país más “feliz” del 2021 por continente según la data?

df_1 = df.select('Regional indicator','Country name','Ladder score').orderBy(['Regional indicator', 'Ladder score'], ascending = [1,0])

df_2 = df.groupBy('Regional indicator').agg(max('Ladder score').alias('Max Ladder Score'))

df_3 = df_1.join(df_2, df_1['Ladder score'] == df_2['Max Ladder score'])

df_3.select(df_1['Regional indicator'],'Country name','Ladder score').show()

+--------------------+--------------------+------------+
|  Regional indicator|        Country name|Ladder score|
+--------------------+--------------------+------------+
|      Western Europe|             Finland|       7.842|
|North America and...|         New Zealand|       7.277|
|Middle East and N...|              Israel|       7.157|
|Latin America and...|          Costa Rica|       7.069|
|Central and Easte...|      Czech Republic|       6.965|
|           East Asia|Taiwan Province o...|       6.584|
|      Southeast Asia|           Singapore|       6.377|
|Commonwealth of I...|          Uzbekistan|       6.179|
|  Sub-Saharan Africa|           Mauritius|       6.049|
|          South Asia|               Nepal|       5.269|
+--------------------+--------------------+------------+



In [4]:
# 3. ¿Cuál es el país que más veces ocupó el primer lugar en todos los años?
df_h = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report.csv")
df_h = df_h.withColumnRenamed('Country name', 'Country_Name').withColumnRenamed('Life Ladder', 'Life_Ladder')
df_h.createOrReplaceTempView("temp_table")

df_h_rankeated = spark.sql("SELECT Country_Name, year, Life_Ladder, RANK() OVER (PARTITION BY year ORDER BY Life_Ladder desc) AS Ranking_Life_Ladder FROM temp_table ORDER BY Ranking_Life_Ladder, year")#.show(16)

df_rank1 = df_h_rankeated.filter(col('Ranking_Life_Ladder') == 1)
df_rank1.show()

freq = df_rank1.groupby('Country_Name').count()
freq.sort(desc('count')).show()

+------------+----+-----------+-------------------+
|Country_Name|year|Life_Ladder|Ranking_Life_Ladder|
+------------+----+-----------+-------------------+
|     Denmark|2005|      8.019|                  1|
|     Finland|2006|      7.672|                  1|
|     Denmark|2007|      7.834|                  1|
|     Denmark|2008|      7.971|                  1|
|     Denmark|2009|      7.683|                  1|
|     Denmark|2010|      7.771|                  1|
|     Denmark|2011|      7.788|                  1|
| Switzerland|2012|      7.776|                  1|
|      Canada|2013|      7.594|                  1|
|     Denmark|2014|      7.508|                  1|
|      Norway|2015|      7.603|                  1|
|     Finland|2016|      7.660|                  1|
|     Finland|2017|      7.788|                  1|
|     Finland|2018|      7.858|                  1|
|     Finland|2019|      7.780|                  1|
|     Finland|2020|      7.889|                  1|
+-----------

In [5]:
# 4. ¿Qué puesto de Felicidad tiene el país con mayor GDP del 2020?
df_h = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report.csv")
df_h = df_h.withColumnRenamed('Country name', 'Country_Name').withColumnRenamed('Life Ladder', 'Life_Ladder').withColumnRenamed('Log GDP per capita', 'Log_GDP_per_capita')
df_h = df_h.withColumn('Log_GDP_per_capita',df_h.Log_GDP_per_capita.cast('double'))
df_h.createOrReplaceTempView("temp_table")

df_h_rankeated = spark.sql("SELECT Country_Name, year, Log_GDP_per_capita, RANK() OVER (PARTITION BY year ORDER BY Life_Ladder desc) AS Ranking_Life_Ladder, RANK() OVER (PARTITION BY year ORDER BY Log_GDP_per_capita desc) AS Ranking_GDP FROM temp_table ORDER BY Ranking_Life_Ladder, year")

df_rank1 = df_h_rankeated.filter(col('Ranking_GDP') == 1)
df_rank1 = df_rank1.filter(col('year') == 2020)
df_rank1.show()

+------------+----+------------------+-------------------+-----------+
|Country_Name|year|Log_GDP_per_capita|Ranking_Life_Ladder|Ranking_GDP|
+------------+----+------------------+-------------------+-----------+
|     Ireland|2020|            11.323|                 13|          1|
+------------+----+------------------+-------------------+-----------+



In [6]:
# 5. ¿En que porcentaje a variado a nivel mundial el GDP promedio del 2020 respecto al 2021? ¿Aumentó o disminuyó?

df = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report-2021.csv")
df = df.withColumnRenamed('Logged GDP per capita', 'Logged_GDP_per_capita')
df = df.withColumn('Logged_GDP_per_capita',df.Logged_GDP_per_capita.cast('double'))

df = df.agg(avg('Logged_GDP_per_capita').alias('Media_GDP2021'))
df.collect()[0]
df.show()

df_h = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report.csv")
df_h = df_h.withColumnRenamed('Country name', 'Country_Name').withColumnRenamed('Life Ladder', 'Life_Ladder').withColumnRenamed('Log GDP per capita', 'Log_GDP_per_capita')
df_h = df_h.withColumn('Log_GDP_per_capita',df_h.Log_GDP_per_capita.cast('double'))

df_h = df_h.groupBy('year').agg(avg('Log_GDP_per_capita').alias('Media_GDP2020'))
df_h = df_h.filter(col('year') == 2020)
df_h = df_h.select('Media_GDP2020')

df_h.show()

GDP2020VSGDP2021 = (1 - (df.collect()[0][0]/ df_h.collect()[0][0]))*100
print("El GDP entre 2020 y 2021 ha disminuido en un", GDP2020VSGDP2021,"%")

+-----------------+
|    Media_GDP2021|
+-----------------+
|9.432208053691273|
+-----------------+

+-----------------+
|    Media_GDP2020|
+-----------------+
|9.751329545454546|
+-----------------+

El GDP entre 2020 y 2021 ha disminuido en un 3.272594678251106 %


In [7]:
# 6. ¿Cuál es el país con mayor expectativa de vide (“Healthy life expectancy at birth”)? Y ¿Cuánto tenia en ese indicador en el 2019?
print("2021")

df = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report-2021.csv")
df = df.withColumnRenamed('Country name', 'Country_Name').withColumnRenamed('Life Ladder', 'Life_Ladder').withColumnRenamed('Healthy life expectancy', 'Healthy_life_expectancy')
df = df.withColumn('Healthy_life_expectancy',df.Healthy_life_expectancy.cast('double'))
df.createOrReplaceTempView("temp_table")

df_rankeated = spark.sql("SELECT Country_Name, Healthy_life_expectancy, RANK() OVER (ORDER BY Healthy_life_expectancy desc) AS Ranking_Life_Expectancy FROM temp_table ORDER BY Ranking_Life_Expectancy")

df_rank1 = df_rankeated.filter(col('Ranking_Life_Expectancy') == 1)
df_rank1.show()

print("2019")
df_h = spark.read.option("header", "true").csv("C:/Users/Victor/Desktop/big-data-processing/big-data-processing Víctor/Práctica/Datasets/world-happiness-report.csv")
df_h = df_h.withColumnRenamed('Country name', 'Country_Name').withColumnRenamed('Life Ladder', 'Life_Ladder').withColumnRenamed('Healthy life expectancy at birth', 'Healthy_life_expectancy_at_birth')
df_h = df_h.withColumn('Healthy_life_expectancy_at_birth',df_h.Healthy_life_expectancy_at_birth.cast('double'))
df_h.createOrReplaceTempView("temp_table")

df_h_rankeated = spark.sql("SELECT Country_Name, year, Healthy_life_expectancy_at_birth, RANK() OVER (PARTITION BY year ORDER BY Healthy_life_expectancy_at_birth desc) AS Ranking_Life_Expectancy FROM temp_table ORDER BY Ranking_Life_Expectancy, year")

df_rank1 = df_h_rankeated.filter(col('Ranking_Life_Expectancy') == 1)
df_rank1 = df_rank1.filter(col('year') == 2019)
df_rank1.show()

2021
+------------+-----------------------+-----------------------+
|Country_Name|Healthy_life_expectancy|Ranking_Life_Expectancy|
+------------+-----------------------+-----------------------+
|   Singapore|                 76.953|                      1|
+------------+-----------------------+-----------------------+

2019
+------------+----+--------------------------------+-----------------------+
|Country_Name|year|Healthy_life_expectancy_at_birth|Ranking_Life_Expectancy|
+------------+----+--------------------------------+-----------------------+
|   Singapore|2019|                            77.1|                      1|
+------------+----+--------------------------------+-----------------------+

